# Tests for DataMux

In [1]:
import logging
logging.basicConfig(format="%(asctime)-15s %(message)s", level=logging.INFO)

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from serving import WebsocketClient

server_host = "0.0.0.0"
server_port = 3300

client = WebsocketClient(server_host, server_port, backend="json")
await client.connect()

2023-07-31 04:20:28,288 Connected to DataMux: ws://0.0.0.0:3300/ws


In [4]:
import asyncio

collections = None
collection_streams = None
live_streams = None

async def log_messages():

    global collections
    global collection_streams
    global live_streams

    while True:
        topic, content = await client.recvmsg()
        if topic == client.protocol.LIST_COLLECTIONS:
            collections = content["collections"]
        if topic == client.protocol.LIST_COLLECTION_STREAMS:
            collection_streams = content["streams"]
        if topic == client.protocol.LIST_LIVE_STREAMS:
            live_streams = content["streams"]
logging_task = asyncio.create_task(log_messages())

In [5]:
await client.sendmsg(*client.datamux.list_collections())

2023-07-31 04:20:40,724 topic: b'list_collections'
2023-07-31 04:20:40,726 content: {'collections': [{'name': 'adhd_sin', 'description': 'Eye movements of participants during an audiovisual-SIN task', 'keywords': ['SIN', 'ADHD'], 'authors': [{'name': 'Anne M. P. Michalek', 'affiliation': 'Old Dominion University', 'email': 'aperrott@odu.edu'}], 'streams': {'gaze': {'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {}}, 'pupil': {'name': 'Pupil', 'description': 'pupil diameter', 'unit': 'mm', 'frequency': 60.0, 'fields': {'d': {'name': 'pupil d', 'des

In [6]:
await asyncio.sleep(2)
assert collections
await client.sendmsg(*client.datamux.list_collection_streams(collections[0]["name"]))

In [7]:
await asyncio.sleep(2)
assert collections
assert collection_streams
await client.sendmsg(*client.datamux.replay_collection_stream(
    collections[0]["name"],
    collection_streams[0]["name"],
    collection_streams[0]["attrs"]
))

2023-07-31 04:20:47,610 topic: b'list_collection_streams'
2023-07-31 04:20:47,630 content: {'collection_name': 'adhd_sin', 'streams': [{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze'}}, {'name': 'Pupil', 'description': 'pupil diameter', 'unit': 'mm', 'frequency': 60.0, 'fields': {'d': {'name': 'pupil d', 'description': 'Pupil Diameter', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recor

2023-07-31 04:20:49,446 topic: b'replay_collection_stream'
2023-07-31 04:20:49,448 content: {'collection_name': 'adhd_sin', 'stream_name': 'Gaze', 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze'}, 'status': 1}
2023-07-31 04:20:49,450 topic: b'data'
2023-07-31 04:20:49,451 content: {'index': [0.0], 'value': [973.0, 748.0]}
2023-07-31 04:20:49,460 topic: b'data'
2023-07-31 04:20:49,461 content: {'index': [16.670000076293945], 'value': [976.0, 752.0]}
2023-07-31 04:20:49,478 topic: b'data'
2023-07-31 04:20:49,479 content: {'index': [33.33000183105469], 'value': [975.0, 750.0]}
2023-07-31 04:20:49,495 topic: b'data'
2023-07-31 04:20:49,496 content: {'index': [50.0], 'value': [973.0, 748.0]}
2023-07-31 04:20:49,513 topic: b'data'
2023-07-31 04:20:49,514 content: {'index': [66.66000366210938], 'value': [972.0, 745.0]}
2023-07-31 04:20:49,531 topic: b'data'
2023-07-31 04:20:49,532 content: {'ind

In [8]:
await asyncio.sleep(2)
assert collections
assert collection_streams
await client.sendmsg(*client.datamux.restream_collection_stream(
    collections[0]["name"],
    collection_streams[0]["name"],
    collection_streams[0]["attrs"]
))

2023-07-31 04:21:00,986 topic: b'restream_collection_stream'
2023-07-31 04:21:00,987 content: {'collection_name': 'adhd_sin', 'stream_name': 'Gaze', 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze'}, 'status': 1}


In [9]:
await client.sendmsg(*client.datamux.list_live_streams())

2023-07-31 04:21:14,005 topic: b'list_live_streams'
2023-07-31 04:21:14,006 content: {'streams': [{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze', 'dfds_mode': 'restream'}}]}


In [10]:
await asyncio.sleep(2)
assert live_streams
await client.sendmsg(*client.datamux.relay_live_streams(
    live_streams[0]["name"],
    live_streams[0]["attrs"]
))

2023-07-31 04:21:22,081 topic: b'relay_live_stream'
2023-07-31 04:21:22,082 content: {'stream_name': 'Gaze', 'status': 1}


2023-07-31 04:21:22,100 topic: b'data'
2023-07-31 04:21:22,102 content: {'index': [744517.202457756], 'value': [[973.0, 748.0]]}
2023-07-31 04:21:22,118 topic: b'data'
2023-07-31 04:21:22,119 content: {'index': [16.670000076293945], 'value': [[976.0, 752.0]]}
2023-07-31 04:21:22,135 topic: b'data'
2023-07-31 04:21:22,137 content: {'index': [33.33000183105469], 'value': [[975.0, 750.0]]}
2023-07-31 04:21:22,153 topic: b'data'
2023-07-31 04:21:22,154 content: {'index': [50.0], 'value': [[973.0, 748.0]]}
2023-07-31 04:21:22,170 topic: b'data'
2023-07-31 04:21:22,171 content: {'index': [66.66000366210938], 'value': [[972.0, 745.0]]}
2023-07-31 04:21:22,187 topic: b'data'
2023-07-31 04:21:22,189 content: {'index': [83.33000183105469], 'value': [[966.0, 747.0]]}
2023-07-31 04:21:22,205 topic: b'data'
2023-07-31 04:21:22,206 content: {'index': [99.95999908447266], 'value': [[970.0, 753.0]]}
2023-07-31 04:21:22,223 topic: b'data'
2023-07-31 04:21:22,224 content: {'index': [116.62999725341797],